In [ ]:
import networkx as nx
import networkx.algorithms.community as nxcom
import matplotlib.pyplot as plt
import numpy as np
import time as time
from random import uniform, seed
from igraph import *
import copy
import pandas as pd
from matplotlib import figure
import random as rd

In [ ]:
import import_ipynb
import Models as models

In [ ]:
# default probability is 0.5 and monte carlo is 10
# g is the graph, k is the seed size
# # https://github.com/hautahi/IM_GreedyCELF  
def celf(g,k,p=0.5,mc=10, model = "IC"):  
    if(model == 'IC'):
        model = models.Independent_Cascade()
    elif(model == 'DC'):
        model = models.Decreasing_Cascade()
    elif(model == 'TR'):
        model = models.Trivalency_Model()
    elif(model == 'WC'):
        model = models.Weighted_Cascade()
    elif(model == 'LT'):
        model = models.linear_threshold()
    elif(model == 'GT'):
        model = models.general_threshold()
    start_time = time.time() 
    marg_gain_m = []
    if(model == 'IC' or model == 'DC' or model == 'TR' or model == 'WC'):
        for node in g.edges():
            marg_gain_m.append(model.monte_carlo_diffusion([node], num_simulations = mc))
    elif(model == 'LT' or model == 'GT'):
        for node in g.nodes():
            marg_gain_m.append(model.diffusion_all([node], k))
    Q = sorted(zip(range(g.vcount()),marg_gain_m), key=lambda x: x[1],reverse=True) 
    seeds, spread, spreads_list = [Q[0][0]], Q[0][1], [Q[0][1]]
    Q, count_of_look_ups, times = Q[1:], [g.vcount()], [time.time()-start_time]
    for i in range(k-1): 
        check  = False
        node_lookup = 0
        while not check:
            node_lookup += 1
            current = Q[0][0]
            if(model == 'IC' or model == 'DC' or model == 'TR' or model == 'WC'):
                Q[0] = (current, model.monte_carlo_diffusion_step([node], num_simulations = mc) - spread)
            elif(model == 'LT' or model == 'GT'):
                Q[0] = (current, model.diffusion_step([node], k) - spread)
            Q = sorted(Q, key = lambda x: x[1], reverse = True)
            check = (Q[0][0] == current)
        spread += Q[0][1]
        seeds.append(Q[0][0])
        spreads_list.append(spread)
        count_of_look_ups.append(node_lookup)
        times.append(time.time() - start_time)
        Q = Q[1:]

    return(seeds,spreads_list,times,count_of_look_ups)